### Step1 - 导入相关包 & 初始化设置

In [ ]:
import os, sys

import numpy as np

from langchain_community.document_loaders import PyPDFLoader
# https://reference.langchain.com/python/langchain_text_splitters/
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma

# 默读取当前目录下的 .env 文件, 可以通过 dotenv_path 来修改
from dotenv import load_dotenv
root_dir = os.path.dirname(os.getcwd())
load_dotenv(dotenv_path=os.path.join(root_dir, ".env"))

# 记录日志
from loguru import logger
logger.remove()
logger.add(sys.stderr, level=os.getenv("LOG_LEVEL"))

1

### Step2 - 加载 PDF 文档

In [ ]:
# 利用 PyPDFLoader 实例对象来读取指定路径的 pdf 文件
file_path = "./Attention Is All You Need.pdf"
loader = PyPDFLoader(file_path)
docs = loader.load()

logger.debug(f"整个文档对象的类型: {type(docs)}")
logger.debug(f"每个文档对象的类型: {type(docs[0])}")
# 文档内容为 docs[0].page_content
logger.debug(f"每个文档对象元数据: {docs[0].metadata}")
logger.debug(f"文档总页数: {len(docs)}")

2025-12-07 14:14:50.518 | DEBUG    | __main__:<module>:6 - 整个文档对象的类型: <class 'list'>
2025-12-07 14:14:50.519 | DEBUG    | __main__:<module>:7 - 每个文档对象的类型: <class 'langchain_core.documents.base.Document'>
2025-12-07 14:14:50.520 | DEBUG    | __main__:<module>:9 - 每个文档对象元数据: {'producer': 'pdfTeX-1.40.17', 'creator': 'LaTeX with hyperref package', 'creationdate': '2017-12-07T01:03:15+00:00', 'author': '', 'keywords': '', 'moddate': '2017-12-07T01:03:15+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.17 (TeX Live 2016) kpathsea version 6.2.2', 'subject': '', 'title': '', 'trapped': '/False', 'source': './1706.03762v5.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1'}
2025-12-07 14:14:50.520 | DEBUG    | __main__:<module>:10 - 文档总页数: 15


### Step3 - 切分文档

In [3]:
# 创建一个文本切分器, 将文档切分为块(chunk)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    add_start_index=True,
)
# 切分指定文档
# all_chunks 是一个列表, 每个元素是一个 Document 对象
all_chunks = text_splitter.split_documents(docs)
# Document 内容为 all_chunks[0].page_content
logger.debug(f"切分后每个块元数据: {all_chunks[0].metadata}")

2025-12-07 14:14:50.529 | DEBUG    | __main__:<module>:11 - 切分后每个块元数据: {'producer': 'pdfTeX-1.40.17', 'creator': 'LaTeX with hyperref package', 'creationdate': '2017-12-07T01:03:15+00:00', 'author': '', 'keywords': '', 'moddate': '2017-12-07T01:03:15+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.17 (TeX Live 2016) kpathsea version 6.2.2', 'subject': '', 'title': '', 'trapped': '/False', 'source': './1706.03762v5.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1', 'start_index': 0}


### Step4 - 将切分后的文档块向量化

In [4]:
# 创建一个嵌入式模型, 这里使用 Ollama 嵌入模型来进行文本向量化
embed_model = OllamaEmbeddings(
    model=os.getenv("OLLAMA_EMB_MODEL"),
    base_url=os.getenv("OLLAMA_BASE_URL"),
)
# 可以通过 embed_query() 方法将单个文本向量化
# 可以通过 embed_documents() 方法将多个文本向量化
logger.debug(f"单个文本经过向量化后的维度: {len(embed_model.embed_query(all_chunks[0].page_content))}")
logger.debug(f"多个文本经过向量化后的形状: {np.array(embed_model.embed_documents(["doc_test1", "doc_test2"])).shape}")

2025-12-07 14:14:53.841 | DEBUG    | __main__:<module>:8 - 单个文本经过向量化后的维度: 2560
2025-12-07 14:14:54.097 | DEBUG    | __main__:<module>:9 - 多个文本经过向量化后的形状: (2, 2560)


### Step5 - 保存向量化后的数据

In [5]:
# 需要创建向量数据库, 这里使用 Chroma 来进行存储
vector_db = Chroma(
    # 集合名字
    # 可以通过创建一个 chromadb.PersistentClient(Path) 对象, 并通过 list_collections() 方法来获得所有集合的名字
    collection_name="example_collection",
    embedding_function=embed_model,
    persist_directory="./chroma_db",
    # 可选配置, 涉及如何计算相似度, Chroma 默认使用的是 L2 距离
    # https://docs.trychroma.com/docs/collections/configure#hnsw-index-configuration
    # https://reference.langchain.com/python/integrations/langchain_chroma/
    # https://reference.langchain.com/python/integrations/langchain_chroma/#langchain_chroma.Chroma.as_retriever
    collection_metadata={"hnsw:space": "l2"}
)
# 将切分好的文档添加到向量数据库中
ids = vector_db.add_documents(all_chunks)
logger.info(f"已成功添加 {len(ids)} 个文档分块的记录")

2025-12-07 14:14:59.213 | INFO     | __main__:<module>:16 - 已成功添加 52 个文档分块的记录
